In [1]:
import pandas as pd
import glob
import csv
import sys
csv.field_size_limit(2147483647)

file_path = "datasets/*.csv"  # Adjust to your dataset folder
file_list = glob.glob(file_path)

df_list = []

for file in file_list:
    try:
        df = pd.read_csv(file, encoding="utf-8", on_bad_lines="skip", engine="python")  
        df_list.append(df)
    except Exception as e:
        print(f"❌ Error reading {file}: {e}")

df = pd.concat(df_list, ignore_index=True)
print(f"✅ Loaded {len(file_list)} CSV files with {df.shape[0]} total samples.")





✅ Loaded 6 CSV files with 187336 total samples.


In [2]:
chunks = []
for file in file_list:
    try:
        for chunk in pd.read_csv(file, encoding="utf-8", on_bad_lines="skip", engine="python", chunksize=10000):
            chunks.append(chunk)
    except Exception as e:
        print(f"❌ Error reading {file}: {e}")

df = pd.concat(chunks, ignore_index=True)
print(f"✅ Successfully loaded CSVs with {df.shape[0]} samples.")

✅ Successfully loaded CSVs with 187336 samples.


In [3]:
for file in file_list:
    try:
        df = pd.read_csv(file, encoding="utf-8",)  # Read only first 5 rows
        print(f"📂 {file} Columns: {df.columns.tolist()}")
    except Exception as e:
        print(f"❌ Error in {file}: {e}")

📂 datasets\Assassin.csv Columns: ['sender', 'receiver', 'date', 'subject', 'body', 'label', 'urls']
📂 datasets\CEAS-08.csv Columns: ['sender', 'receiver', 'date', 'subject', 'body', 'label', 'urls']
📂 datasets\Enron.csv Columns: ['subject', 'body', 'label']
📂 datasets\Ling.csv Columns: ['subject', 'body', 'label']
❌ Error in datasets\TREC-05.csv: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.

📂 datasets\TREC-07.csv Columns: ['sender', 'receiver', 'date', 'subject', 'body', 'label', 'urls']


In [4]:
df.to_csv("cleaned_file12.csv", encoding="utf-8", index=False)

In [5]:
import re

def clean_text(text):
    text = str(text)  # Convert to string in case of NaN values
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r"<.*?>", "", text)  # Remove HTML tags
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    return text


In [6]:
# Apply Cleaning to Email Body
df['cleaned_text'] = df['body'].apply(clean_text) 

In [7]:
import pandas as pd
import numpy as np
import re
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:



# 🔹 Train-Test Split (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 🔹 Train Random Forest Model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

KeyboardInterrupt: 

In [23]:
# 🔹 TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['cleaned_text']).toarray()
y = df['label']  # Ensure CSV has a 'label' column (0 = legitimate, 1 = phishing)
# Sample data
documents = ["This is a phishing email", "Click this link to win a prize", "Legitimate business email"]


# Fit and transform the data
tfidf_matrix = vectorizer.fit_transform(documents)

In [29]:
import xgboost as xgb

# Create DMatrix (XGBoost’s data structure)
dtrain = xgb.DMatrix(X_train, label=y_train)

# Set parameters to use GPU
params = {
    'tree_method': 'CUDA',  # Use GPU acceleration
    'predictor': 'gpu_predictor',
    'objective': 'binary:logistic'
}

# Train model
model = xgb.train(params, dtrain, num_boost_round=100)

# Make predictions
preds = model.predict(xgb.DMatrix(X_test))


XGBoostError: Invalid Input: 'CUDA', valid values are: {'approx', 'auto', 'exact', 'gpu_hist', 'hist'}

In [17]:
def evaluate_model(model_name, y_true, y_pred):
    print(f"\n🔍 {model_name} Performance:")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred):.4f}")
    print(f"F1 Score: {f1_score(y_true, y_pred):.4f}")
evaluate_model("XGBoost", y_test, y_pred_xgb)


🔍 XGBoost Performance:
Accuracy: 0.9899
Precision: 0.9867
Recall: 0.9949
F1 Score: 0.9908


In [ ]:
# ##from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV

# # Define parameter grid
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [10, 20, None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }

# # Perform Grid Search
# rf = RandomForestClassifier(random_state=42)
# grid_search = GridSearchCV(rf, param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)
# grid_search.fit(X_train, y_train)

# # Get Best Model
# best_rf = grid_search.best_estimator_
# print("Best RF Parameters:", grid_search.best_params_)


Fitting 3 folds for each of 81 candidates, totalling 243 fits


In [11]:
#### 🔹 Predictions
y_pred_rf = rf_model.predict(X_test)


# 🔹 Model Evaluation
def evaluate_model(model_name, y_true, y_pred):
    print(f"\n🔍 {model_name} Performance:")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred):.4f}")
    print(f"F1 Score: {f1_score(y_true, y_pred):.4f}")

evaluate_model("Random Forest", y_test, y_pred_rf)####


🔍 Random Forest Performance:
Accuracy: 0.9904
Precision: 0.9892
Recall: 0.9934
F1 Score: 0.9913


In [24]:
import pickle

with open("model.pkl1", "wb") as f:
    pickle.dump(rf_model, f)  # Replace with your trained model variable

with open("vectorizer.pkl2", "wb") as f:
    pickle.dump(vectorizer, f)  # Replace with your trained TF-IDF vectorizer


In [5]:
import gradio as gr
import pandas as pd
import re
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

# 🔹 Load Trained Model & Vectorizer
with open("pickle/model.pkl1", "rb") as f:
    model = pickle.load(f)

with open("pickle/vectorizer.pkl2", "rb") as f:
    vectorizer = pickle.load(f)

# 🔹 Text Cleaning Function
def clean_text(text):
    text = str(text)  
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # Remove URLs
    text = re.sub(r"<.*?>", "", text)  # Remove HTML tags
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    return text

# 🔹 Prediction Function
def predict_email(subject, body):
    email_text = subject + " " + body  # Combine subject & body
    cleaned_text = clean_text(email_text)
    vectorized_text = vectorizer.transform([cleaned_text])  # TF-IDF Vectorization
    prediction = model.predict(vectorized_text)[0]
    return "🔴 Phishing Email" if prediction == 1 else "🟢 Legitimate Email"

# 🔹 Gradio UI
with gr.Blocks() as ui:
    gr.Markdown("## 🚀 Phishing Email Detection")
    
    with gr.Row():
        subject = gr.Textbox(label="📩 Email Subject")
        body = gr.Textbox(label="✉️ Email Body", lines=5)
    
    predict_btn = gr.Button("🔍 Detect")
    result = gr.Textbox(label="Result", interactive=False)
    
    predict_btn.click(predict_email, inputs=[subject, body], outputs=result)

# 🔹 Run App
ui.launch()


* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


In [26]:
import torch
print(torch.cuda.is_available())  # Should print True
print(torch.cuda.get_device_name(0))  # Should print your GPU name

False


AssertionError: Torch not compiled with CUDA enabled